# Using onset/duration format

Many datasets store interval annotations as onset + duration rather than explicit interval boundaries. This notebook shows how to use `DimensionInterval` with onset/duration coordinates directly, without needing to construct `pd.IntervalIndex` objects.

## Why onset/duration?

Common data formats like TextGrid, Praat, and many annotation tools export intervals as:
- `onset`: when the interval starts
- `duration`: how long the interval lasts

The `linked_indices` library provides helper functions to convert annotation DataFrames directly to xarray coordinates with proper naming conventions.

In [1]:
import xarray as xr
from linked_indices import DimensionInterval, example_data
from linked_indices.example_data import (
    intervals_from_dataframe,
    intervals_from_long_dataframe,
)

## Loading annotation data

Annotation data typically comes as a pandas DataFrame with onset, duration, and label columns. Let's load some example speech annotation data:

In [2]:
# Load example speech annotations (simulating data from Praat, TextGrid, etc.)
annotations = example_data.speech_annotations()
annotations

,onset,duration,word
0,0.5,1.2,hello
1,2.1,1.8,world
2,4.5,2.0,how
3,7.0,2.5,are


Notice that the annotations have **gaps** between them - this is common in real speech data where there are pauses between words. For example, "hello" ends at 1.7s but "world" doesn't start until 2.1s.

## Converting DataFrame to xarray coordinates

The `intervals_from_dataframe` function converts annotation DataFrames to xarray Datasets with properly named coordinates (`{dim}_onset`, `{dim}_duration`):

In [3]:
# Convert annotations DataFrame to xarray coordinates
word_coords = intervals_from_dataframe(annotations, dim_name="word", label_col="word")
word_coords

<xarray.Dataset> Size: 96B
Dimensions:        (word: 4)
Coordinates:
  * word           (word) object 32B 'hello' 'world' 'how' 'are'
    word_onset     (word) float64 32B 0.5 2.1 4.5 7.0
    word_duration  (word) float64 32B 1.2 1.8 2.0 2.5
Data variables:
    *empty*

The helper automatically creates:
- `word` as the dimension coordinate (from `label_col`)
- `word_onset` and `word_duration` as coordinates (named `{dim}_onset`, `{dim}_duration`)

## Adding audio data

Now we can add our continuous audio signal and merge with the annotation coordinates:

In [4]:
# Generate a simulated audio signal
times, audio_signal = example_data.generate_audio_signal(duration=10.0)

# Create dataset by merging annotation coordinates with audio data
ds = word_coords.copy()
ds["audio"] = (("time",), audio_signal)
ds = ds.assign_coords(time=times)
ds

<xarray.Dataset> Size: 16kB
Dimensions:        (word: 4, time: 1000)
Coordinates:
  * word           (word) object 32B 'hello' 'world' 'how' 'are'
  * time           (time) float64 8kB 0.0 0.01001 0.02002 ... 9.98 9.99 10.0
    word_onset     (word) float64 32B 0.5 2.1 4.5 7.0
    word_duration  (word) float64 32B 1.2 1.8 2.0 2.5
Data variables:
    audio          (time) float64 8kB 0.09934 0.3325 0.7231 ... -0.4744 0.1145

## Applying the DimensionInterval index

To link the time and word dimensions, apply `DimensionInterval` with the `onset_duration_coords` option mapping dimension names to `(onset_coord, duration_coord)` tuples:

In [5]:
ds = ds.drop_indexes(["time", "word"]).set_xindex(
    ["time", "word_onset", "word_duration", "word"],
    DimensionInterval,
    onset_duration_coords={"word": ("word_onset", "word_duration")},
)
ds

<xarray.Dataset> Size: 16kB
Dimensions:        (word: 4, time: 1000)
Coordinates:
  * word           (word) object 32B 'hello' 'world' 'how' 'are'
  * time           (time) float64 8kB 0.0 0.01001 0.02002 ... 9.98 9.99 10.0
  * word_onset     (word) float64 32B 0.5 2.1 4.5 7.0
  * word_duration  (word) float64 32B 1.2 1.8 2.0 2.5
Data variables:
    audio          (time) float64 8kB 0.09934 0.3325 0.7231 ... -0.4744 0.1145
Indexes:
  ┌ time           DimensionInterval
  │ word_onset
  │ word_duration
  └ word

Notice that:
- The `word_onset` and `word_duration` coordinates remain visible
- All coordinates are linked under a single `DimensionInterval` index
- No manual coordinate creation was needed - the helper handled naming conventions

## Selecting data

Selection works exactly the same as with the IntervalIndex format. When you select on any dimension, all other dimensions are automatically constrained.

In [6]:
# Select by word label - time is automatically constrained
ds.sel(word="hello")

<xarray.Dataset> Size: 2kB
Dimensions:        (time: 120, word: 1)
Coordinates:
  * time           (time) float64 960B 0.5005 0.5105 0.5205 ... 1.682 1.692
  * word           (word) object 8B 'hello'
  * word_onset     (word) float64 8B 0.5
  * word_duration  (word) float64 8B 1.2
Data variables:
    audio          (time) float64 960B 0.06827 -0.0255 -0.1359 ... 0.4946 0.2634
Indexes:
  ┌ time           DimensionInterval
  │ word_onset
  │ word_duration
  └ word

In [7]:
# Select by time range - words are automatically constrained
ds.sel(time=slice(2, 5))

<xarray.Dataset> Size: 5kB
Dimensions:        (time: 300, word: 2)
Coordinates:
  * time           (time) float64 2kB 2.002 2.012 2.022 ... 4.975 4.985 4.995
  * word           (word) object 16B 'world' 'how'
  * word_onset     (word) float64 16B 2.1 4.5
  * word_duration  (word) float64 16B 1.8 2.0
Data variables:
    audio          (time) float64 2kB 0.1481 0.5324 0.8344 ... -0.6739 -0.4654
Indexes:
  ┌ time           DimensionInterval
  │ word_onset
  │ word_duration
  └ word

In [8]:
# Select by onset value
ds.sel(word_onset=4.5)

<xarray.Dataset> Size: 3kB
Dimensions:        (time: 200, word: 1)
Coordinates:
  * time           (time) float64 2kB 4.505 4.515 4.525 ... 6.476 6.486 6.496
  * word           (word) object 8B 'how'
  * word_onset     (word) float64 8B 4.5
  * word_duration  (word) float64 8B 2.0
Data variables:
    audio          (time) float64 2kB 0.01689 0.2374 -0.2569 ... -0.0957 0.1158
Indexes:
  ┌ time           DimensionInterval
  │ word_onset
  │ word_duration
  └ word

## Handling gaps

Our word annotations have gaps between them (silence between words). Let's see what happens when we select time in a gap:

In [9]:
# Time 1.8 to 2.0 is in the gap between "hello" (ends at 1.7) and "world" (starts at 2.1)
ds.sel(time=slice(1.75, 2.0))

<xarray.Dataset> Size: 424B
Dimensions:        (time: 25, word: 1)
Coordinates:
  * time           (time) float64 200B 1.752 1.762 1.772 ... 1.972 1.982 1.992
  * word           (word) object 8B 'hello'
  * word_onset     (word) float64 8B 0.5
  * word_duration  (word) float64 8B 1.2
Data variables:
    audio          (time) float64 200B -0.2943 -0.2311 ... -0.5498 -0.5234
Indexes:
  ┌ time           DimensionInterval
  │ word_onset
  │ word_duration
  └ word

When selecting multiple words with gaps between them using `isel`, the time dimension spans the **union** of their intervals (including the gap). Here we select "hello" [0.5, 1.7) and "world" [2.1, 3.9):

In [10]:
# Select first two words - time spans from 0.5 to 3.9, including the gap
ds.isel(word=slice(0, 2))

<xarray.Dataset> Size: 11kB
Dimensions:        (time: 340, word: 2)
Coordinates:
  * time           (time) float64 3kB 0.5005 0.5105 0.5205 ... 3.874 3.884 3.894
  * word           (word) object 16B 'hello' 'world'
  * word_onset     (word) float64 16B 0.5 2.1
  * word_duration  (word) float64 16B 1.2 1.8
Data variables:
    audio          (time) float64 8kB 0.09934 0.3325 0.7231 ... -0.4744 0.1145
Indexes:
  ┌ time           DimensionInterval
  │ word_onset
  │ word_duration
  └ word

## Multiple onset/duration dimensions

You can have multiple interval dimensions, each with their own onset/duration coordinates. This is common for hierarchical annotations like words and phonemes. The helper function makes it easy to convert each level:

In [11]:
# Load multi-level annotations (words and phonemes)
word_annotations, phoneme_annotations = example_data.multi_level_annotations()

display(word_annotations)
display(phoneme_annotations)

# Convert each DataFrame to xarray coordinates using helpers
word_ds = intervals_from_dataframe(word_annotations, dim_name="word", label_col="word")
phoneme_ds = intervals_from_dataframe(
    phoneme_annotations, dim_name="phoneme", label_col="phoneme"
)

,onset,duration,word,part_of_speech
0,0.0,2.5,hello,interjection
1,3.0,2.5,world,noun
2,6.0,3.5,test,noun


,onset,duration,phoneme
0,0.0,0.8,hh
1,0.8,0.9,eh
2,1.7,0.8,l
3,3.0,0.9,w
4,3.9,0.8,er
5,4.7,0.8,ld
6,6.0,1.2,t
7,7.2,2.3,st


In [12]:
# Merge annotation coordinates and add audio data
times, audio = example_data.generate_audio_signal(duration=10.0)

ds_multi = xr.merge([word_ds, phoneme_ds])
ds_multi["audio"] = (("time",), audio)
ds_multi = ds_multi.assign_coords(time=times)

# Apply index with both onset/duration mappings
ds_multi = ds_multi.drop_indexes(["time", "word", "phoneme"]).set_xindex(
    [
        "time",
        "word_onset",
        "word_duration",
        "word",
        "part_of_speech",
        "phoneme_onset",
        "phoneme_duration",
        "phoneme",
    ],
    DimensionInterval,
    onset_duration_coords={
        "word": ("word_onset", "word_duration"),
        "phoneme": ("phoneme_onset", "phoneme_duration"),
    },
)
ds_multi

<xarray.Dataset> Size: 16kB
Dimensions:           (word: 3, phoneme: 8, time: 1000)
Coordinates:
  * word              (word) object 24B 'hello' 'world' 'test'
  * phoneme           (phoneme) object 64B 'hh' 'eh' 'l' 'w' 'er' 'ld' 't' 'st'
  * time              (time) float64 8kB 0.0 0.01001 0.02002 ... 9.98 9.99 10.0
  * word_onset        (word) float64 24B 0.0 3.0 6.0
  * word_duration     (word) float64 24B 2.5 2.5 3.5
  * part_of_speech    (word) object 24B 'interjection' 'noun' 'noun'
  * phoneme_onset     (phoneme) float64 64B 0.0 0.8 1.7 3.0 3.9 4.7 6.0 7.2
  * phoneme_duration  (phoneme) float64 64B 0.8 0.9 0.8 0.9 0.8 0.8 1.2 2.3
Data variables:
    audio             (time) float64 8kB 0.09934 0.3325 ... -0.4744 0.1145
Indexes:
  ┌ time              DimensionInterval
  │ word_onset
  │ word_duration
  │ word
  │ part_of_speech
  │ phoneme_onset
  │ phoneme_duration
  └ phoneme

In [13]:
# Select word "hello" - both time AND phonemes are constrained
ds_multi.sel(word="hello")

<xarray.Dataset> Size: 4kB
Dimensions:           (time: 250, word: 1, phoneme: 3)
Coordinates:
  * time              (time) float64 2kB 0.0 0.01001 0.02002 ... 2.482 2.492
  * word              (word) object 8B 'hello'
  * phoneme           (phoneme) object 24B 'hh' 'eh' 'l'
  * word_onset        (word) float64 8B 0.0
  * word_duration     (word) float64 8B 2.5
  * part_of_speech    (word) object 8B 'interjection'
  * phoneme_onset     (phoneme) float64 24B 0.0 0.8 1.7
  * phoneme_duration  (phoneme) float64 24B 0.8 0.9 0.8
Data variables:
    audio             (time) float64 2kB 0.09934 0.3325 ... 0.3239 0.03694
Indexes:
  ┌ time              DimensionInterval
  │ word_onset
  │ word_duration
  │ word
  │ part_of_speech
  │ phoneme_onset
  │ phoneme_duration
  └ phoneme

In [14]:
# Select by part of speech - finds all nouns
ds_multi.sel(part_of_speech="noun")

<xarray.Dataset> Size: 11kB
Dimensions:           (time: 650, word: 2, phoneme: 5)
Coordinates:
  * time              (time) float64 5kB 3.003 3.013 3.023 ... 9.479 9.489 9.499
  * word              (word) object 16B 'world' 'test'
  * phoneme           (phoneme) object 40B 'w' 'er' 'ld' 't' 'st'
  * word_onset        (word) float64 16B 3.0 6.0
  * word_duration     (word) float64 16B 2.5 3.5
  * part_of_speech    (word) object 16B 'noun' 'noun'
  * phoneme_onset     (phoneme) float64 40B 3.0 3.9 4.7 6.0 7.2
  * phoneme_duration  (phoneme) float64 40B 0.9 0.8 0.8 1.2 2.3
Data variables:
    audio             (time) float64 5kB -0.05135 0.3361 ... -0.06063 0.04521
Indexes:
  ┌ time              DimensionInterval
  │ word_onset
  │ word_duration
  │ word
  │ part_of_speech
  │ phoneme_onset
  │ phoneme_duration
  └ phoneme

## Controlling interval closedness

By default, intervals are left-closed `[onset, onset+duration)`. You can change this with the `interval_closed` option:

In [15]:
# Reload data for fresh example
annotations = example_data.speech_annotations()
times, audio = example_data.generate_audio_signal()

# Create with right-closed intervals (onset, onset+duration]
ds_right = xr.Dataset(
    {"audio": (("time",), audio)},
    coords={
        "time": times,
        "word_onset": ("word", annotations["onset"].values),
        "word_duration": ("word", annotations["duration"].values),
        "word": ("word", annotations["word"].values),
    },
)

ds_right = ds_right.drop_indexes(["time", "word"]).set_xindex(
    ["time", "word_onset", "word_duration", "word"],
    DimensionInterval,
    onset_duration_coords={"word": ("word_onset", "word_duration")},
    interval_closed="right",  # Options: "left", "right", "both", "neither"
)
print("Created dataset with right-closed intervals (onset, onset+duration]")

Created dataset with right-closed intervals (onset, onset+duration]


## Summary

The onset/duration format provides a convenient way to work with interval data without manually constructing `pd.IntervalIndex` objects:

1. **Load annotations** as a pandas DataFrame (from TextGrid, Praat, CSV, etc.)
2. **Convert to coordinates** using `intervals_from_dataframe()` or `intervals_from_long_dataframe()`
3. **Merge and add data** - combine annotation coordinates with your continuous data
4. **Apply the index** with `onset_duration_coords` mapping
5. **Select data** - all selection operations work identically to IntervalIndex format

### Helper functions

| Function | Use case |
|----------|----------|
| `intervals_from_dataframe()` | Convert a single-event-type DataFrame |
| `intervals_from_long_dataframe()` | Convert a multi-event-type DataFrame with category column |

### Key features

- **Natural representation**: Use onset + duration directly from your data files
- **Library helpers**: Handle coordinate naming conventions automatically
- **Visible coordinates**: onset and duration remain as regular coordinates  
- **Full functionality**: All selection operations work identically
- **Multiple dimensions**: Support for multiple onset/duration pairs
- **Gap support**: Non-contiguous intervals work correctly
- **Mixed events**: Handle DataFrames with multiple event types

## Handling multiple event types in one DataFrame

Sometimes annotation data comes as a single "long format" DataFrame with multiple event types (words, phonemes, stimuli, etc.) distinguished by a category column. The `intervals_from_long_dataframe` function handles this case:

In [16]:
# Load example mixed-event annotations
mixed_df = example_data.mixed_event_annotations()
mixed_df

,onset,duration,label,event_type
0,0.0,2.5,hello,word
1,3.0,2.5,world,word
2,6.0,3.5,test,word
3,0.0,0.8,hh,phoneme
4,0.8,0.9,eh,phoneme
5,1.7,0.8,l,phoneme
6,3.0,0.9,w,phoneme
7,3.9,1.6,orld,phoneme
8,0.0,5.0,image_A,stimulus
9,5.0,5.0,image_B,stimulus


In [17]:
# Convert all event types at once
intervals_from_long_dataframe(mixed_df)

<xarray.Dataset> Size: 240B
Dimensions:            (word: 3, phoneme: 5, stimulus: 2)
Coordinates:
  * word               (word) object 24B 'hello' 'world' 'test'
  * phoneme            (phoneme) object 40B 'hh' 'eh' 'l' 'w' 'orld'
  * stimulus           (stimulus) object 16B 'image_A' 'image_B'
    word_onset         (word) float64 24B 0.0 3.0 6.0
    word_duration      (word) float64 24B 2.5 2.5 3.5
    phoneme_onset      (phoneme) float64 40B 0.0 0.8 1.7 3.0 3.9
    phoneme_duration   (phoneme) float64 40B 0.8 0.9 0.8 0.9 1.6
    stimulus_onset     (stimulus) float64 16B 0.0 5.0
    stimulus_duration  (stimulus) float64 16B 5.0 5.0
Data variables:
    *empty*

In [18]:
# Add time/audio and apply DimensionInterval
times, audio = example_data.generate_audio_signal(duration=10.0)
interval_ds = intervals_from_long_dataframe(mixed_df)

ds_mixed = interval_ds.copy()
ds_mixed["audio"] = (("time",), audio)
ds_mixed = ds_mixed.assign_coords(time=times)

# Apply the index with all three event types
ds_mixed = ds_mixed.drop_indexes(["time", "word", "phoneme", "stimulus"]).set_xindex(
    [
        "time",
        "word_onset",
        "word_duration",
        "word",
        "phoneme_onset",
        "phoneme_duration",
        "phoneme",
        "stimulus_onset",
        "stimulus_duration",
        "stimulus",
    ],
    DimensionInterval,
    onset_duration_coords={
        "word": ("word_onset", "word_duration"),
        "phoneme": ("phoneme_onset", "phoneme_duration"),
        "stimulus": ("stimulus_onset", "stimulus_duration"),
    },
)
ds_mixed

<xarray.Dataset> Size: 16kB
Dimensions:            (word: 3, phoneme: 5, stimulus: 2, time: 1000)
Coordinates:
  * word               (word) object 24B 'hello' 'world' 'test'
  * phoneme            (phoneme) object 40B 'hh' 'eh' 'l' 'w' 'orld'
  * stimulus           (stimulus) object 16B 'image_A' 'image_B'
  * time               (time) float64 8kB 0.0 0.01001 0.02002 ... 9.98 9.99 10.0
  * word_onset         (word) float64 24B 0.0 3.0 6.0
  * word_duration      (word) float64 24B 2.5 2.5 3.5
  * phoneme_onset      (phoneme) float64 40B 0.0 0.8 1.7 3.0 3.9
  * phoneme_duration   (phoneme) float64 40B 0.8 0.9 0.8 0.9 1.6
  * stimulus_onset     (stimulus) float64 16B 0.0 5.0
  * stimulus_duration  (stimulus) float64 16B 5.0 5.0
Data variables:
    audio              (time) float64 8kB 0.09934 0.3325 ... -0.4744 0.1145
Indexes:
  ┌ time               DimensionInterval
  │ word_onset
  │ word_duration
  │ word
  │ phoneme_onset
  │ phoneme_duration
  │ phoneme
  │ stimulus_onset
  │ stimulus_duration
  └ stimulus

In [19]:
# Selecting a stimulus constrains words and phonemes too
ds_mixed.sel(stimulus="image_A")

<xarray.Dataset> Size: 8kB
Dimensions:            (time: 500, word: 2, phoneme: 5, stimulus: 1)
Coordinates:
  * time               (time) float64 4kB 0.0 0.01001 0.02002 ... 4.985 4.995
  * word               (word) object 16B 'hello' 'world'
  * phoneme            (phoneme) object 40B 'hh' 'eh' 'l' 'w' 'orld'
  * stimulus           (stimulus) object 8B 'image_A'
  * word_onset         (word) float64 16B 0.0 3.0
  * word_duration      (word) float64 16B 2.5 2.5
  * phoneme_onset      (phoneme) float64 40B 0.0 0.8 1.7 3.0 3.9
  * phoneme_duration   (phoneme) float64 40B 0.8 0.9 0.8 0.9 1.6
  * stimulus_onset     (stimulus) float64 8B 0.0
  * stimulus_duration  (stimulus) float64 8B 5.0
Data variables:
    audio              (time) float64 4kB 0.09934 0.3325 ... -0.6739 -0.4654
Indexes:
  ┌ time               DimensionInterval
  │ word_onset
  │ word_duration
  │ word
  │ phoneme_onset
  │ phoneme_duration
  │ phoneme
  │ stimulus_onset
  │ stimulus_duration
  └ stimulus

### Manual iteration for selective event types

If you only want some event types, you can filter and apply `intervals_from_dataframe` iteratively:

In [20]:
# Only include words and phonemes (exclude stimuli)
datasets = []
for event_type in ["word", "phoneme"]:
    subset = mixed_df[mixed_df["event_type"] == event_type].drop(columns=["event_type"])
    ds_subset = intervals_from_dataframe(subset, dim_name=event_type, label_col="label")
    datasets.append(ds_subset)

xr.merge(datasets)

<xarray.Dataset> Size: 192B
Dimensions:           (word: 3, phoneme: 5)
Coordinates:
  * word              (word) object 24B 'hello' 'world' 'test'
  * phoneme           (phoneme) object 40B 'hh' 'eh' 'l' 'w' 'orld'
    word_onset        (word) float64 24B 0.0 3.0 6.0
    word_duration     (word) float64 24B 2.5 2.5 3.5
    phoneme_onset     (phoneme) float64 40B 0.0 0.8 1.7 3.0 3.9
    phoneme_duration  (phoneme) float64 40B 0.8 0.9 0.8 0.9 1.6
Data variables:
    *empty*